In [1]:
#Load libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

### Logistic Regression with Experimental Data

In [2]:
#Load experimental data 
E4031_before = pd.read_csv('experiment_data/E4031_before.dat', header=None)
E4031_after = pd.read_csv('experiment_data/E4031_after.dat', header=None)
Flecainide_before = pd.read_csv('experiment_data/Flecainide_before.dat', header=None)
Flecainide_after = pd.read_csv('experiment_data/Flecainide_after.dat', header=None)
Nifedipine_before = pd.read_csv('experiment_data/Nifedipine_before.dat', header=None)
Nifedipine_after = pd.read_csv('experiment_data/Nifedipine_after.dat', header=None)
TTX_before = pd.read_csv('experiment_data/TTX-mold2-before.dat', header=None)
TTX_after = pd.read_csv('experiment_data/TTX-mold2-after.dat', header=None)
col_names = pd.read_csv('experiment_data/DataLabels.txt').columns.tolist()
col_names = [col[1:] for col in col_names]

In [3]:
#Calculate metric percentage differences after compound administration
E4031_diff = (E4031_after - E4031_before)/E4031_before
E4031_diff.columns = col_names[:-1]
Flecainide_diff = (Flecainide_after - Flecainide_before)/Flecainide_before
Flecainide_diff.columns = col_names[:-1]
Nifedipine_diff = (Nifedipine_after - Nifedipine_before)/Nifedipine_before
Nifedipine_diff.columns = col_names[:-1]
TTX_diff = (TTX_after - TTX_before)/TTX_before
TTX_diff.columns = col_names[:-1]

#Adjust column labeling
E4031_diff['pharm'] = 'E4031'
TTX_diff['pharm'] = 'TTX'
Flecainide_diff['pharm'] = 'Flec'
Nifedipine_diff['pharm'] = 'Nif'

E4031_diff['channel_blocked'] = 'kr'
TTX_diff['channel_blocked'] = 'na'
Flecainide_diff['channel_blocked']  = 'nakr'
Nifedipine_diff['channel_blocked']  = 'ca'

#Concatenate datasets for all compounds
all_data_exp = pd.concat([E4031_diff, Nifedipine_diff, TTX_diff, Flecainide_diff])


In [4]:
#Predictor variable names
X_col = ['APD30', 'APD50', 'APD80', 'APDmxr', 'tri', 'rise time', 'stim delay',]

#Containers for results
test_scores = []
channel_scores_exp = [[] for i in range(4)]

for i in range(100):
    #Divide training and test datasets
    train_exp, test_exp = train_test_split(all_data_exp, test_size = 0.2, stratify = all_data_exp.channel_blocked)
    
    #Separate predictors and response variables
    X_train_exp, y_train_exp = train_exp[X_col], train_exp['channel_blocked']
    X_test_exp, y_test_exp = test_exp[X_col], test_exp['channel_blocked']
    
    #Find scaler using training dataset then apply to test dataset
    scaler_exp = StandardScaler().fit(X_train_exp)
    scaled_X_train_exp = scaler_exp.transform(X_train_exp)
    scaled_X_test_exp = scaler_exp.transform(X_test_exp)
    
    #Cross-validate to tune model parameters
    clf_exp = LogisticRegressionCV(Cs = 5, cv = 5, multi_class = 'multinomial').fit(scaled_X_train_exp, y_train_exp)
    
    #Save results 
    test_scores.append(clf_exp.score(scaled_X_test_exp, y_test_exp))
    
    #Obtain model performance for individual ion channels
    for idx, channel in enumerate(['kr', 'ca', 'na', 'nakr']):
        num = np.sum(clf_exp.predict(scaled_X_test_exp)[np.array(y_test_exp == channel)]  == channel)
        denom = np.sum(y_test_exp == channel)
        channel_scores_exp[idx].append(num/denom)

In [5]:
# Print results
print("Mean Model Accuracy on Full Dataset: ", np.mean(test_scores))
print("Standard Deviation of Model Accuracy on Full Dataset: ", np.std(test_scores))
print()

print("Model Accuracy for Individual Ion Channels:")
pd.DataFrame([np.mean(score) for score in channel_scores_exp], index = ['kr', 'ca', 'na', 'nakr'],
            columns = ['Model Accuracy'])

Mean Model Accuracy on Full Dataset:  0.9467857142857141
Standard Deviation of Model Accuracy on Full Dataset:  0.03829763940079687

Model Accuracy for Individual Ion Channels:


,Model Accuracy
kr,0.951429
ca,0.971429
na,0.921429
nakr,0.942857


### Logistic Regression with Simulation Data- 5 Ion Channels

In [6]:
all_sims = pd.read_excel('dfsfromcardiotoxsims/fixedrisetimes25/sim_df_joined.xlsx')

In [7]:
all_sims = all_sims[all_sims['APDmxr'] < 2*all_sims['APD80']]

In [ ]:
#Reserve test set
X_col = ['APD30', 'APD50', 'APD80', 'APDmxr', 'APDtri', 'rise_time', 'delay',]

test_scores_sim = []
channel_scores = [[] for i in range(5)]

for i in range(5):
    #Divide training and test datasets
    train, test = train_test_split(all_sims, test_size = 0.2, stratify = all_sims.channel_blocked)
    
    #Separate predictors and response variables
    X_train, y_train = train[X_col], train['channel_blocked']
    X_test, y_test = test[X_col], test['channel_blocked']
    
    #Find scaler using training dataset then apply to test dataset
    scaler = StandardScaler().fit(X_train)
    scaled_X_train = scaler.transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    #Cross-validate to tune model parameters 
    clf = LogisticRegressionCV(Cs = 3, cv = 5, solver = 'sag', penalty = 'l2', 
                           multi_class = 'multinomial', max_iter = 300, 
                          tol = 1e-3).fit(scaled_X_train, y_train)

    #Save results 
    test_scores_sim.append(clf.score(scaled_X_test, y_test))
    
    #Obtain model performance for individual ion channels
    for idx, channel in enumerate(['kr', 'ca', 'na','ks', 'to']):
        num = np.sum(clf.predict(scaled_X_test)[y_test == channel]  == channel)
        denom = np.sum(y_test == channel)
        channel_scores[idx].append(num/denom)
    

In [ ]:
# Print results
print("Mean Model Accuracy on Full Dataset: ", np.mean(test_scores))
print("Standard Deviation of Model Accuracy on Full Dataset: ", np.std(test_scores))
print()

print("Model Accuracy for Individual Ion Channels:")
pd.DataFrame([[np.mean(score) for score in channel_scores], [np.std(score) for score in channel_scores]], 
             columns = ['kr', 'ca', 'na', 'ks', 'to'],
            index = ['Model Accuracy', 'Model SD']).T

### Logistic Regression with Simulation Data- IKr, ICa, INa Only (Mimics Experimental Data)

In [ ]:
all_sims_short = all_sims[all_sims.channel_blocked != 'to']
all_sims_short = all_sims_short[all_sims_short.channel_blocked != 'ks']

#Reserve test set
X_col = ['APD30', 'APD50', 'APD80', 'APDmxr', 'APDtri', 'rise_time', 'delay',]

test_scores_sim_short = []
channel_scores_short = [[] for i in range(5)]

for i in range(5):
    #Divide training and test datasets
    train, test = train_test_split(all_sims_short, test_size = 0.2, stratify = all_sims_short.channel_blocked)
    
    #Separate predictors and response variables
    X_train, y_train = train[X_col], train['channel_blocked']
    X_test, y_test = test[X_col], test['channel_blocked']
    
    #Find scaler using training dataset then apply to test dataset
    scaler = StandardScaler().fit(X_train)
    scaled_X_train = scaler.transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    #Cross-validate to tune model parameters 
    clf = LogisticRegressionCV(Cs = 3, cv = 5, solver = 'sag', penalty = 'l2', 
                           multi_class = 'multinomial', max_iter = 300, 
                          tol = 1e-3).fit(scaled_X_train, y_train)

    #Save results 
    test_scores_sim_short.append(clf.score(scaled_X_test, y_test))
    
    #Obtain model performance for individual ion channels
    for idx, channel in enumerate(['kr', 'ca', 'na']):
        num = np.sum(clf.predict(scaled_X_test)[y_test == channel]  == channel)
        denom = np.sum(y_test == channel)
        channel_scores_short[idx].append(num/denom)
    

In [ ]:
# Print results
print("Mean Model Accuracy on Full Dataset: ", np.mean(test_scores_sim_short))
print("Standard Deviation of Model Accuracy on Full Dataset: ", np.std(test_scores_sim_short))
print()

print("Model Accuracy for Individual Ion Channels:")
pd.DataFrame([[np.mean(score) for score in channel_scores_short[:3]], [np.std(score) for score in channel_scores_short[:3]]], 
             columns = ['kr', 'ca', 'na'],
            index = ['Model Accuracy', 'Model SD']).T